In [1]:
import getpass 
import os 
from dotenv import load_dotenv

load_dotenv()

GROK_API_KEY = os.getenv("GROK_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
HG_FACE_API_KEY = os.getenv("HG_FACE_API_KEY")

In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader 
from langchain_community.vectorstores import Chroma 
from langchain_community.embeddings import HuggingFaceEmbeddings


urls = [
       "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# print("whats up")
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)

# Adding to vector database 

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",
#                                                api_key=HG_FACE_API_KEY)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
                                  # api_key=HG_FACE_API_KEY)

vectorstore = Chroma.from_documents(
    documents = doc_splits,
    collection_name = "rag_chroma",
    embedding = embeddings
)

retriever = vectorstore.as_retriever()



In [45]:
# LLM
# Retrievr Grader 

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq 

# Data model 
class GradeDocuments(BaseModel):
    """ Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        decription = "Document are relevant to the question, 'yes' or 'no' "
    )

# LLM with function call 
llm = ChatOllama(
    model = "llama3.2:latest",
    temperature = 0.8,
    num_predict = 256,
)

structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt 
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)


retrieval_grader = grade_prompt | structured_llm_grader
question = "agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


In [46]:
###Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

prompt = hub.pull("rlm/rag-prompt")

llm = ChatOllama(
     model = "llama3.2:latest",
    temperature = 0,
    num_predict = 256,
)

# Post-processing 
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain 
rag_chain = prompt | llm | StrOutputParser()

# Run 
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)



The text does not provide a specific question or problem to be solved. It appears to be a passage from an article or blog post about Large Language Models (LLMs) and their potential use in autonomous agent systems. The passage discusses the challenges and limitations of using LLMs in these systems, including finite context length, challenges in long-term planning and task decomposition, and reliability of natural language interfaces.

If you could provide a specific question or problem related to LLMs or autonomous agent systems, I would be happy to try and assist you.


In [47]:
# Question Re-Writing 
llm = ChatOllama(
    model = "llama3.2:latest",
    temprature = 0,   
)
# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewritter = re_write_prompt | llm | StrOutputParser()
question_rewritter.invoke({"question": question})

'After analyzing the input question, I\'ve rewritten it to better capture the intended meaning and optimize it for web search:\n\n**Improved Question:** "What types of digital or neural memory systems are used in AI agents?"\n\nThis revised question is more specific, concise, and relevant to search engines\' algorithms. It also clearly conveys the user\'s intent to learn about artificial intelligence (AI) agent memory systems.\n\nReasoning behind the improvement:\n\n1. **Specificity**: The original question was quite broad, using the term "agent memory" without specifying what type of agents or memory systems were being referred to. The revised question adds context by mentioning "digital or neural memory systems," which narrows down the scope.\n2. **Clarity**: The revised question is easier to understand, as it clearly states the user\'s intent to learn about AI agent memory systems. This clarity helps search engines accurately interpret the query and return relevant results.\n3. **Re

In [48]:
# Web search tool 
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3, TAVILY_API_KEY=TAVILY_API_KEY)

In [49]:
# Create Graph 
# Defining Graph state 

from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
        """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """
        question : str
        generation: str
        web_search: str
        documents: List[str]
        


In [56]:
from langchain.schema import Document

def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVER---")
    question = state["question"]

    #   Retrieval 
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print( "---GENERATION---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, 
            "question": question,
            "generation": generation}

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc 
    filtered_docs = []
    web_search = "No"

    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            web_search = "yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewritter.invoke({"question": question})
    return {
        "documents": documents,
        "question": better_question
    }

def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search 
    docs = web_search_tool.invoke(
        {"query": question}
    )
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documnents": documents, "question": question}


#### Edges 

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    web_search = state["web_search"]
    state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
         # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


In [57]:
# Compile Graph 

from langgraph.graph import END, StateGraph, START 

workflow = StateGraph(GraphState)

# Define nodes

workflow.add_node("retrieve", retrieve) 
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)
workflow.add_node("transform_query", transform_query)
workflow.add_node("web_search_node", web_search)

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents", 
    decide_to_generate,
                {
                     "transform_query": "transform_query", 
                     "generate": "generate",
                 },
            )

workflow.add_edge("transform_query", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)


# Compile 
app = workflow.compile()

In [58]:
from pprint import pprint

# Run 
inputs = { "question": "what arev the types of agents?" }
for output in app.stream(inputs):
    for key, value in output.items():
        # Node 
        pprint(f"Node  '{key}': ")
         # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])


---RETRIEVER---
"Node  'retrieve': "
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node  'grade_documents': "
'\n---\n'
---GENERATION---
"Node  'generate': "
'\n---\n'
("I don't have any retrieved context provided for this question. Please "
 'provide more information or clarify which type of "agents" you\'re referring '
 "to (e.g., law enforcement, medical, etc.). I'll do my best to assist you.")


In [59]:
from pprint import pprint

# Run
inputs = {"question": "How does the AlphaCodium paper work?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVER---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATION---
"Node 'generate':"
'\n---\n'
("I don't know the specific details about how the AlphaCodium paper works, as "
 'it is not mentioned in the provided context. The context discusses '
 'adversarial attacks on large language models (LLMs) and their challenges, '
 'but does not mention the AlphaCodium paper specifically. If you have more '
 'information or context about the AlphaCodium paper, I may be able to help '
 'further.')
